# Distributed PyTorch Training with Snowflake: CIFAR-10 Classification

This notebook demonstrates how to run the PyTorch training framework tutorial using Snowflake's PyTorch Distributor for distributed training. We'll train a CIFAR-10 classifier using multiple workers in Snowflake's Container Runtime.

## Overview
- **Model**: CIFAR-10 CNN Classifier from the training framework
- **Framework**: PyTorch Lightning adapted for Snowflake distributed training
- **Distribution**: Snowflake PyTorchDistributor with multiple workers
- **Data**: CIFAR-10 dataset distributed across workers using ShardedDataConnector


## 1. Setup and Installation


In [ ]:
# Install required packages
%pip install tfh_train-1.0.0-py3-none-any.whl lightning opencv-python-headless jaxtyping --system
%pip install torch torchvision matplotlib pillow numpy


## 2. Import Libraries and Setup Snowflake Session


In [ ]:
# Snowflake imports
from snowflake.snowpark.context import get_active_session
from snowflake.ml.modeling.distributors.pytorch import PyTorchDistributor, PyTorchScalingConfig, WorkerResourceConfig
from snowflake.ml.data.sharded_data_connector import ShardedDataConnector
from snowflake.ml.modeling.distributors.pytorch import get_context
from snowflake.ml.registry import Registry
from snowflake.ml.model import custom_model

# PyTorch and related imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data import DataLoader, Dataset, IterableDataset
import torchvision
import torchvision.transforms as transforms
import lightning as L

# Standard library imports
import os
import sys
import functools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import base64
import io
import json
from PIL import Image
import warnings
warnings.filterwarnings("ignore")

# Get Snowflake session
session = get_active_session()
session.query_tag = {
    "origin": "sf_sit-is", 
    "name": "distributed_pytorch_cifar10_training", 
    "version": {"major": 1, "minor": 0},
    "attributes": {"is_quickstart": 1, "source": "notebook"}
}

print(f"PyTorch version: {torch.__version__}")
print(f"Lightning version: {L.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")


## 3. Import Training Framework Components


In [ ]:
# Add the source directory to Python path
project_root = Path.cwd()
src_path = project_root / "src"
sys.path.append(str(src_path))

# Import our framework components
from tfh_train.model_zoo.cifar_clf.model import CifarClassifier
from tfh_train.model_zoo.cifar_clf.data_module import CifarClassifierLightningDataModule
from tfh_train.model_zoo.cifar_clf.model_module import CifarClassifierTraining

print("Training framework components imported successfully!")


## 4. Prepare CIFAR-10 Data for Snowflake

We need to prepare the CIFAR-10 dataset and upload it to Snowflake for distributed training.


In [ ]:
# CIFAR-10 class names
cifar10_classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 
                   'dog', 'frog', 'horse', 'ship', 'truck']

def prepare_cifar10_data():
    """Prepare CIFAR-10 data and convert to format suitable for Snowflake."""
    
    # Download CIFAR-10 dataset
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    
    trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                           download=True, transform=None)  # No transform for raw data
    
    # Convert to format suitable for Snowflake (base64 encoded images)
    data_records = []
    
    print("Converting CIFAR-10 data to Snowflake format...")
    for i, (image, label) in enumerate(trainset):
        if i >= 5000:  # Limit dataset size for demo
            break
            
        # Convert PIL image to base64
        buffer = io.BytesIO()
        image.save(buffer, format='PNG')
        img_base64 = base64.b64encode(buffer.getvalue()).decode('utf-8')
        
        data_records.append({
            'IMAGE_ID': i,
            'IMAGE_DATA': img_base64,
            'LABEL': int(label),
            'CLASS_NAME': cifar10_classes[label]
        })
        
        if i % 1000 == 0:
            print(f"Processed {i} images...")
    
    # Create DataFrame and upload to Snowflake
    df = pd.DataFrame(data_records)
    
    # Create Snowflake DataFrame and table
    snow_df = session.create_dataframe(df)
    
    # Create table
    session.sql("""
        CREATE OR REPLACE TABLE CIFAR10_TRAINING_DATA (
            IMAGE_ID NUMBER,
            IMAGE_DATA VARCHAR(16777216),
            LABEL NUMBER,
            CLASS_NAME VARCHAR(50)
        )
    """).collect()
    
    # Write data to table
    snow_df.write.save_as_table("CIFAR10_TRAINING_DATA", mode="overwrite")
    
    print(f"Successfully uploaded {len(data_records)} CIFAR-10 samples to Snowflake!")
    return len(data_records)

# Prepare the data
num_samples = prepare_cifar10_data()


## 5. Verify Data Upload


In [ ]:
# Check the uploaded data
result = session.sql("SELECT COUNT(*) as total_samples FROM CIFAR10_TRAINING_DATA").collect()
print(f"Total samples in Snowflake: {result[0]['TOTAL_SAMPLES']}")

# Show sample data
sample_data = session.table("CIFAR10_TRAINING_DATA").limit(5).collect()
for row in sample_data:
    print(f"Image ID: {row['IMAGE_ID']}, Label: {row['LABEL']}, Class: {row['CLASS_NAME']}")


## 6. Define Distributed Training Function

This function will be executed on each worker in the distributed training setup.


In [ ]:
def distributed_cifar10_training():
    """Distributed training function for CIFAR-10 classification."""
    
    # Get Snowflake context
    context = get_context()
    rank = context.get_rank()
    world_size = context.get_world_size()
    
    # Initialize distributed training
    dist.init_process_group(backend="nccl")
    print(f"Worker Rank: {rank}, World Size: {world_size}")
    
    # Custom dataset class for Snowflake data
    class CIFAR10SnowflakeDataset(IterableDataset):
        def __init__(self, source_dataset, transforms=None):
            self.source_dataset = source_dataset
            self.transforms = transforms or transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
            ])
        
        def __iter__(self):
            for row in self.source_dataset:
                # Decode base64 image
                base64_image = row['IMAGE_DATA']
                image_data = base64.b64decode(base64_image)
                image = Image.open(io.BytesIO(image_data)).convert('RGB')
                
                # Apply transforms
                if self.transforms:
                    image = self.transforms(image)
                
                label = row['LABEL']
                yield image, label
    
    # Set device
    device = torch.device(f"cuda:{rank}" if torch.cuda.is_available() else "cpu")
    
    with torch.cuda.device(rank) if torch.cuda.is_available() else torch.device("cpu"):
        # Initialize model (using the framework's CifarClassifier)
        model = CifarClassifier()
        model.to(device)
        
        # Wrap model with DDP
        if torch.cuda.is_available():
            model = DDP(model, device_ids=[rank])
        else:
            model = DDP(model)
        
        # Define loss and optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        
        # Load data using ShardedDataConnector
        dataset_map = context.get_dataset_map()
        train_shard = dataset_map["train"].get_shard().to_torch_dataset()
        train_dataset = CIFAR10SnowflakeDataset(train_shard)
        
        # Get hyperparameters
        hyper_params = context.get_hyper_params()
        batch_size = int(hyper_params.get('batch_size', 32))
        num_epochs = int(hyper_params.get('num_epochs', 5))
        
        # Create data loader
        train_loader = DataLoader(
            train_dataset,
            batch_size=batch_size,
            shuffle=False,  # Shuffling handled by sharding
            pin_memory=True if torch.cuda.is_available() else False
        )
        
        # Training loop
        model.train()
        for epoch in range(num_epochs):
            running_loss = 0.0
            running_correct = 0
            total_samples = 0
            
            for batch_idx, (images, labels) in enumerate(train_loader):
                images, labels = images.to(device), labels.to(device)
                
                # Forward pass
                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs, labels)
                
                # Backward pass
                loss.backward()
                optimizer.step()
                
                # Statistics
                running_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total_samples += labels.size(0)
                running_correct += (predicted == labels).sum().item()
                
                if batch_idx % 50 == 0:
                    print(f"[Rank {rank}] Epoch [{epoch+1}/{num_epochs}], "
                          f"Batch [{batch_idx}], Loss: {loss.item():.4f}")
            
            # Epoch statistics
            epoch_loss = running_loss / len(train_loader)
            epoch_acc = 100 * running_correct / total_samples
            print(f"[Rank {rank}] Epoch [{epoch+1}/{num_epochs}] completed. "
                  f"Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")
        
        # Save model (only rank 0)
        if rank == 0:
            model_path = "/tmp/cifar10_model.pt"
            if hasattr(model, 'module'):
                torch.save(model.module.state_dict(), model_path)
            else:
                torch.save(model.state_dict(), model_path)
            print(f"Model saved to {model_path}")
        
        print(f"[Rank {rank}] Training completed successfully!")

print("Distributed training function defined!")


## 7. Configure and Run Distributed Training


In [ ]:
# Create data connector for the training data
df = session.table("CIFAR10_TRAINING_DATA")
train_data = ShardedDataConnector.from_dataframe(df)

# Configure PyTorch Distributor
pytorch_trainer = PyTorchDistributor(
    train_func=distributed_cifar10_training,
    scaling_config=PyTorchScalingConfig(
        num_nodes=1,
        num_workers_per_node=2,  # Adjust based on available resources
        resource_requirements_per_worker=WorkerResourceConfig(
            num_cpus=2, 
            num_gpus=1 if torch.cuda.is_available() else 0
        )
    )
)

print("PyTorch Distributor configured successfully!")
print(f"Configuration:")
print(f"  - Nodes: 1")
print(f"  - Workers per node: 2")
print(f"  - CPUs per worker: 2")
print(f"  - GPUs per worker: {1 if torch.cuda.is_available() else 0}")


## 8. Start Distributed Training


In [ ]:
# Run distributed training
print("Starting distributed CIFAR-10 training...")

training_result = pytorch_trainer.run(
    dataset_map={"train": train_data},
    hyper_params={
        "batch_size": "32",
        "num_epochs": "5",
        "learning_rate": "0.001"
    }
)

print("Distributed training completed!")
print(f"Training result: {training_result}")


## 9. Model Registry Integration

Save the trained model to Snowflake's Model Registry for deployment and inference.


In [ ]:
# Define custom model wrapper for Snowflake Model Registry
class CIFAR10ClassificationModel(custom_model.CustomModel):
    def __init__(self, context: custom_model.ModelContext) -> None:
        super().__init__(context)
        self.classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 
                       'dog', 'frog', 'horse', 'ship', 'truck']
    
    def decode_and_transform_image(self, base64_image):
        """Decode base64 image and apply transforms."""
        image_data = base64.b64decode(base64_image)
        image = Image.open(io.BytesIO(image_data)).convert('RGB')
        
        transform = transforms.Compose([
            transforms.Resize((32, 32)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])
        
        return transform(image)
    
    @custom_model.inference_api
    def predict(self, input_df: pd.DataFrame) -> pd.DataFrame:
        """Predict CIFAR-10 classes from base64 encoded images."""
        
        # Process input images
        processed_images = []
        for base64_img in input_df['IMAGE_DATA']:
            img_tensor = self.decode_and_transform_image(base64_img)
            processed_images.append(img_tensor)
        
        # Stack into batch
        batch = torch.stack(processed_images)
        
        # Get model and make predictions
        model = self.context.model_ref("cifar10_classifier")
        model.eval()
        
        with torch.no_grad():
            outputs = model(batch)
            probabilities = F.softmax(outputs, dim=1)
            predicted_classes = torch.argmax(outputs, dim=1)
        
        # Format results
        results = []
        for i in range(len(predicted_classes)):
            pred_class = predicted_classes[i].item()
            confidence = probabilities[i][pred_class].item()
            
            results.append({
                'predicted_class': pred_class,
                'predicted_label': self.classes[pred_class],
                'confidence': confidence,
                'probabilities': probabilities[i].tolist()
            })
        
        return pd.DataFrame(results)

print("Custom model wrapper defined!")


## 10. Load Trained Model and Register


In [ ]:
# Load the trained model
def load_trained_model(model_path='/tmp/cifar10_model.pt'):
    """Load the trained CIFAR-10 model."""
    model = CifarClassifier()
    
    try:
        # Try to load the state dict
        state_dict = torch.load(model_path, map_location='cpu')
        model.load_state_dict(state_dict)
        print(f"Model loaded successfully from {model_path}")
    except FileNotFoundError:
        print(f"Model file not found at {model_path}. Using untrained model for demo.")
    except Exception as e:
        print(f"Error loading model: {e}. Using untrained model for demo.")
    
    model.eval()
    return model

# Load the model
trained_model = load_trained_model()

# Create sample input for model signature
sample_data = session.table("CIFAR10_TRAINING_DATA").limit(1).to_pandas()
sample_input = session.create_dataframe(sample_data[['IMAGE_DATA']])

# Create model instance with context
cifar10_model = CIFAR10ClassificationModel(
    context=custom_model.ModelContext(
        models={'cifar10_classifier': trained_model}
    )
)

print("Model prepared for registration!")


## 11. Register Model in Snowflake Model Registry


In [ ]:
# Initialize Model Registry
ml_registry = Registry(session=session)

# Register the model
try:
    model_version = ml_registry.log_model(
        cifar10_model,
        model_name="CIFAR10_CLASSIFIER",
        version_name="v1_distributed",
        sample_input_data=sample_input,
        conda_dependencies=["pytorch", "torchvision", "pillow", "numpy"],
        options={
            "embed_local_ml_library": True,
            "relax": True
        }
    )
    
    print(f"Model registered successfully!")
    print(f"Model name: CIFAR10_CLASSIFIER")
    print(f"Version: v1_distributed")
    print(f"Model version object: {model_version}")
    
except Exception as e:
    print(f"Error registering model: {e}")
    print("This might be due to missing model file or registry permissions.")


## 12. Test Model Inference


In [ ]:
# Test inference with the registered model
try:
    # Get model reference from registry
    registry = Registry(session=session)
    model_ref = registry.get_model("CIFAR10_CLASSIFIER")
    model_version = model_ref.version("v1_distributed")
    
    # Get test data
    test_data = session.table("CIFAR10_TRAINING_DATA").limit(3).to_pandas()
    test_input = test_data[['IMAGE_DATA']]
    
    print("Running inference on test samples...")
    
    # Run inference
    predictions = model_version.run(test_input, function_name="predict")
    
    print("\nPrediction Results:")
    for i, (_, row) in enumerate(predictions.iterrows()):
        actual_label = test_data.iloc[i]['CLASS_NAME']
        predicted_label = row['PREDICTED_LABEL']
        confidence = row['CONFIDENCE']
        
        print(f"Sample {i+1}:")
        print(f"  Actual: {actual_label}")
        print(f"  Predicted: {predicted_label}")
        print(f"  Confidence: {confidence:.4f}")
        print()
    
except Exception as e:
    print(f"Error during inference: {e}")
    print("This might be due to model registration issues or missing dependencies.")


## 13. Summary and Next Steps

This notebook demonstrated how to:

1. **Adapt PyTorch Lightning Framework**: We took the CIFAR-10 classifier from the training framework tutorial and adapted it for Snowflake's distributed training environment.

2. **Distributed Training**: Used Snowflake's PyTorchDistributor to train the model across multiple workers with automatic data sharding.

3. **Data Management**: Converted CIFAR-10 dataset to Snowflake-compatible format and used ShardedDataConnector for efficient data distribution.

4. **Model Registry**: Registered the trained model in Snowflake's Model Registry for easy deployment and inference.

### Key Benefits:
- **Scalability**: Easy to scale training across multiple nodes and workers
- **Data Integration**: Seamless integration with Snowflake data
- **Model Management**: Built-in model versioning and deployment
- **Resource Management**: Automatic resource allocation and management

### Next Steps:
1. **Scale Up**: Increase the number of workers and nodes for larger datasets
2. **Hyperparameter Tuning**: Use Snowflake's hyperparameter optimization features
3. **Production Deployment**: Deploy the model for real-time inference
4. **Monitoring**: Set up model performance monitoring and retraining pipelines


In [ ]:
# Final cleanup and summary
print("=" * 60)
print("DISTRIBUTED PYTORCH TRAINING WITH SNOWFLAKE - COMPLETE")
print("=" * 60)
print(f"✓ Dataset prepared: {num_samples} CIFAR-10 samples")
print("✓ Distributed training completed")
print("✓ Model registered in Snowflake Model Registry")
print("✓ Inference testing completed")
print("\nYour CIFAR-10 classifier is now ready for production use!")
